In [30]:
#Set environment variables for your notebook
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/kellywisneski/Downloads/msds434-final-5dd240442926.json'

In [31]:
#Imports google cloud client library and initiates BQ service
from google.cloud import bigquery
from google.cloud import automl_v1beta1
bigquery_client = bigquery.Client()

In [32]:
#Write Query on BQ
query = """
SELECT
    trip_start_timestamp,
    trip_end_timestamp,
    trip_seconds,
    trip_miles,
    pickup_census_tract,
    dropoff_census_tract,
    pickup_community_area,
    dropoff_community_area,
    fare,
    tolls,
    extras,
    trip_total,
    payment_type,
    company,
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude,
    IF((tips/fare >= 0.2),
      1,
      0) AS tip_bin
FROM
    `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    trip_miles > 0
    AND fare > 0
LIMIT
  10000000
  """

In [33]:
#Run the query and write result to a pandas data frame
query_results = bigquery_client.query(query)
df = query_results.to_dataframe()

In [34]:
#View top few rows of result
df.head()

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tolls,extras,trip_total,payment_type,company,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_bin
0,2017-11-19 11:00:00+00:00,2017-11-19 11:15:00+00:00,720,2.3,17031081000,17031281900,8,28,9.75,0.0,0.0,12.25,Credit Card,Globe Taxi,-87.632109,41.900266,-87.642649,41.879255,1
1,2016-06-13 18:00:00+00:00,2016-06-13 18:15:00+00:00,1200,3.6,17031070200,17031281900,7,28,13.75,0.0,1.0,14.75,Cash,City Service,-87.646293,41.929078,-87.642649,41.879255,0
2,2017-11-22 15:45:00+00:00,2017-11-22 15:45:00+00:00,542,1.4,17031838100,17031281900,28,28,7.50,0.0,1.0,8.50,Cash,Taxi Affiliation Service Yellow,-87.671654,41.878667,-87.642649,41.879255,0
3,2017-11-14 16:00:00+00:00,2017-11-14 16:00:00+00:00,230,0.6,17031031400,17031060900,3,6,4.75,0.0,0.0,4.75,Cash,Chicago Carriage Cab Corp,-87.645379,41.962179,-87.643965,41.949829,0
4,2016-06-17 17:00:00+00:00,2016-06-17 17:15:00+00:00,900,1.4,17031060400,17031060900,6,6,9.00,0.0,0.0,9.00,Cash,City Service,-87.666536,41.950673,-87.643965,41.949829,0


In [35]:
#quick summary stats
df.describe()

,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tolls,extras,trip_total,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_bin
count,9.999463e+06,1.000000e+07,6.990160e+06,6.835367e+06,9.574590e+06,8.991285e+06,1.000000e+07,8.751432e+06,1.000000e+07,1.000000e+07,9.574845e+06,9.574845e+06,8.991904e+06,8.991904e+06,1.000000e+07
mean,8.996159e+02,4.877747e+00,1.703138e+10,1.703127e+10,2.615229e+01,2.091420e+01,1.445825e+01,2.180666e-03,1.349599e+00,1.757994e+01,-8.766242e+01,4.189717e+01,-8.764255e+01,4.188987e+01,3.161479e-01
std,1.332485e+03,1.257470e+01,3.373217e+05,2.597870e+05,2.073605e+01,1.596370e+01,3.481857e+01,1.154484e+00,5.835628e+00,3.720892e+01,8.131848e-02,4.160661e-02,5.042823e-02,3.880499e-02,4.649714e-01
min,0.000000e+00,1.000000e-02,1.703101e+10,1.703101e+10,1.000000e+00,1.000000e+00,1.000000e-02,0.000000e+00,0.000000e+00,1.000000e-02,-8.791362e+01,4.166014e+01,-8.791362e+01,4.166014e+01,0.000000e+00
25%,3.910000e+02,1.000000e+00,1.703108e+10,1.703108e+10,8.000000e+00,8.000000e+00,6.250000e+00,0.000000e+00,0.000000e+00,7.500000e+00,-8.765177e+01,4.188099e+01,-8.764265e+01,4.187926e+01,0.000000e+00
50%,6.130000e+02,1.800000e+00,1.703128e+10,1.703124e+10,2.800000e+01,1.400000e+01,8.750000e+00,0.000000e+00,0.000000e+00,1.025000e+01,-8.763275e+01,4.189197e+01,-8.762621e+01,4.189092e+01,0.000000e+00
75%,1.080000e+03,5.100000e+00,1.703184e+10,1.703132e+10,3.200000e+01,3.200000e+01,1.525000e+01,0.000000e+00,1.000000e+00,1.750000e+01,-8.762197e+01,4.190007e+01,-8.761971e+01,4.189833e+01,1.000000e+00
max,8.639600e+04,3.460000e+03,1.703198e+10,1.703198e+10,7.700000e+01,7.700000e+01,9.898980e+03,3.333330e+03,9.243000e+03,9.997960e+03,-8.753139e+01,4.202122e+01,-8.753139e+01,4.202122e+01,1.000000e+00


In [45]:
#create dataset
dataset = bigquery_client.create_dataset("dataset_v1")

In [38]:
#define target column
target_column = 'tip_bin'

In [46]:
#define model query

model_query = f"""
CREATE OR REPLACE MODEL `dataset_v1.tip-model-v1`
OPTIONS(MODEL_TYPE = 'automl_classifier', 
BUDGET_HOURS = 1.0, 
OPTIMIZATION_OBJECTIVE = 'MINIMIZE_LOG_LOSS',
INPUT_LABEL_COLS=['{target_column}'])
AS 
{query}
"""

In [47]:
#create model

model = bigquery_client.query(model_query)
model

QueryJob<project=msds434-final, location=US, id=d25d6854-68e7-4b0d-8c1c-9f70b74ad5ef>

In [48]:
results = model.result()
results